Algoritmo: Q-learning

Código feito com o Google IA.

É interessante que ao deixar um decaimento para o epsilon extremamente alto, ainda assim, após uma centena de iterações, o aprendizado já é o suficiente para vencer o cenário. Eu imagino que isso ocorra pois faz com que o algoritmo ao invés de explorar durante o treino, acaba fazendo como um algoritmo ACO e reforçando as ações positivas e penalizando as negativas e com o tempo passa a tomar somente as positivas.

Também é interessante que o algoritmo só é capaz de encontrar a solução caso haja uma pequena penalidade para cada movimento tomado. Ao deixar como recompensa para o movimento como zero, o algoritmo as vezes chega na solução e as vezes não. Ao deixar positiva, ele faz o que qualquer um faria: fica indo de um lado para o outro infinitamente. Somente ao deixar uma penalidade, mesmo que muito pequena (-0.00000001) por movimento que o algoritmo alcança a solução ideal.

In [12]:
import random
import collections

def preenche_wumpus(tabuleiro, tam_i, tam_j, tam_k, k, a, b):
  # Fixando a posição
  i = 2 # random.randint(0, tam_i - 1)
  j = 1 # random.randint(0, tam_j - 1)
  tabuleiro[i][j][k] = a
  if(i - 1 >= 0): tabuleiro[i-1][j][k] = b
  if(i + 1 < tam_i): tabuleiro[i+1][j][k] = b
  if(j - 1 >= 0): tabuleiro[i][j-1][k] = b
  if(j + 1 < tam_j): tabuleiro[i][j+1][k] = b
  return (i, j)

def edita_wumpus(tabuleiro, tam_i, tam_j, tam_k, i, j, k, a, b):
  tabuleiro[i][j][k] = a
  if(i - 1 >= 0): tabuleiro[i-1][j][k] = b
  if(i + 1 < tam_i): tabuleiro[i+1][j][k] = b
  if(j - 1 >= 0): tabuleiro[i][j-1][k] = b
  if(j + 1 < tam_j): tabuleiro[i][j+1][k] = b

def preenche_abismo(tabuleiro, tam_i, tam_j, tam_k, k, a, b, wumpus_pos):
  # Fixando a posição
  abismos = [(0,1), (2,2), (3,1)]

  abismos_pos = []
  aux = 0
  while len(abismos_pos) < 3 and aux < len(abismos):
      i, j = abismos[aux]
      aux += 1

      tabuleiro[i][j][k] = a
      abismos_pos.append((i,j))
      if(i - 1 >= 0 and tabuleiro[i-1][j][k] != 4):
        tabuleiro[i-1][j][k] = b
      if(i + 1 < tam_i and tabuleiro[i+1][j][k] != 4):
        tabuleiro[i+1][j][k] = b
      if(j - 1 >= 0 and tabuleiro[i][j-1][k] != 4):
        tabuleiro[i][j-1][k] = b
      if(j + 1 < tam_j and tabuleiro[i][j+1][k] != 4):
        tabuleiro[i][j+1][k] = b
  return abismos_pos


def preenche_ouro(tabuleiro, tam_i, tam_j, tam_k, k, a, wumpus_pos, pit_coords_list):
  # Fixando a posição
  i = 3 # random.randint(0, tam_i - 1)
  j = 3 # random.randint(0, tam_j - 1)

  tabuleiro[i][j][k] = a
  return (i,j)

# Implementação do Q-learning
tam_i = 4
tam_j = 4

alpha = 0.1
gamma = 0.9
epsilon = 1.0
epsilon_decay = 0.999
numero_episodios = 5000

# Inicializando a tabela Q cheia de zeros
q_table = collections.defaultdict(lambda: collections.defaultdict(float))

# Definindo as ações
ACTIONS = [
    "mover c", "mover b", "mover e", "mover d",
    "atirar c", "atirar b", "atirar e", "atirar d",
    "pegar ouro",
    "sair caverna"
]

def informacao_estado(pos_i, pos_j, percepcoes, tem_flecha, pegou_ouro):
    return (pos_i, pos_j, percepcoes[0], percepcoes[1], percepcoes[2], tem_flecha, pegou_ouro)

def possiveis_acoes(tem_flecha, pegou_ouro, pos_i, pos_j):
    # Sempre é possível fazer
    available = ["mover c", "mover b", "mover e", "mover d", "pegar ouro"]
    # Caso tenha flecha
    if tem_flecha:
        available.extend(["atirar c", "atirar b", "atirar e", "atirar d"])
    # Caso possa sair da caverna
    if pegou_ouro and pos_i == tam_i - 1 and pos_j == 0:
        available.append("sair caverna")
    return available


def define_acao(estado, tem_flecha, pegou_ouro, pos_i_atual, pos_j_atual, epsilon_atual):
    possiveis_acoes1 = possiveis_acoes(tem_flecha, pegou_ouro, pos_i_atual, pos_j_atual)

    # Verifica se explora ou usa a tabela
    if random.uniform(0, 1) < epsilon_atual:
        return random.choice(possiveis_acoes1)
    else:
        valores_prox = q_table[estado]
        max_q = -float('inf')
        melhor_escolha = None
        for acao in possiveis_acoes1:
            if valores_prox[acao] > max_q:
                max_q = valores_prox[acao]
                melhor_escolha = acao
        return melhor_escolha


def atualizar_tabela(estado, acao, recompensa, prox_estado, done, next_tem_flecha, next_pegou_ouro, next_pos_i, next_pos_j):
    valor_antigo = q_table[estado][acao]

    # Determina o valor máximo para o próximo estado
    next_max_q = 0.0
    if not done:
        next_available_actions = possiveis_acoes(next_tem_flecha, next_pegou_ouro, next_pos_i, next_pos_j)
        if next_available_actions:
            q_values_for_next_state = q_table[prox_estado]
            max_val = -float('inf')
            for act in next_available_actions:
                if q_values_for_next_state[act] > max_val:
                    max_val = q_values_for_next_state[act]
            if max_val != -float('inf'):
                 next_max_q = max_val

    # Formula de atualização
    novo_valor = valor_antigo + alpha * (recompensa + gamma * next_max_q - valor_antigo)
    q_table[estado][acao] = novo_valor

# Loop de treinamento e teste
if True:
    tam_k = 3  # Dimensões: 0:Wumpus, 1:Abismo, 2:Ouro

    recompensa_episodio = []
    num_vitorias = 0

    epsilon_atual = epsilon

    for episodio in range(numero_episodios):
        # Reseta o tabuleiro para cada episódio
        tabuleiro = [[[0 for _ in range(tam_k)] for _ in range(tam_j)] for _ in range(tam_i)]
        # Wumpus = 2, Cheiro ruim = 3
        wumpus_i, wumpus_j = preenche_wumpus(tabuleiro, tam_i, tam_j, tam_k, 0, 2, 3)
        wumpus_pos_original = (wumpus_i, wumpus_j)
        # Abismo = 4, Brisa = 5
        pit_coords_list = preenche_abismo(tabuleiro, tam_i, tam_j, tam_k, 1, 4, 5, wumpus_pos_original)
        # Ouro = 6
        ouro_i, ouro_j = preenche_ouro(tabuleiro, tam_i, tam_j, tam_k, 2, 6, wumpus_pos_original, pit_coords_list)
        ouro_pos_original = (ouro_i, ouro_j)

        # Estado inicial e variáveis
        pos_i = tam_i - 1
        pos_j = 0
        flecha = True
        pegou_ouro = False
        wumpus_morto = False

        # Estatísticas do episódio
        recompensa_atual = 0
        done = False
        iteracao = 0
        iteracoes_por_episodio = 100

        while not done and iteracao < iteracoes_por_episodio:
            # Determina o estado
            cheiro_ruim = (tabuleiro[pos_i][pos_j][0] == 3)
            brisa = (tabuleiro[pos_i][pos_j][1] == 5)
            brilho = (tabuleiro[pos_i][pos_j][2] == 6)
            percepcoes_atual = (cheiro_ruim, brisa, brilho)
            state = informacao_estado(pos_i, pos_j, percepcoes_atual, flecha, pegou_ouro)

            # Tomada da ação
            acao = define_acao(state, flecha, pegou_ouro, pos_i, pos_j, epsilon_atual)

            # Guarda os valores antigos
            pos_i_velha, pos_j_velha = pos_i, pos_j
            flecha_velha, pegou_ouro_velha = flecha, pegou_ouro

            # Simula e recebe a recompensa
            recompensa_teste = -1  # Custo base
            pos_i_prox, pos_j_prox = pos_i, pos_j

            if acao.startswith("mover"):
                direcao = acao.split(" ")[1]
                if direcao == "d": pos_j_prox += 1
                elif direcao == "e": pos_j_prox -= 1
                elif direcao == "c": pos_i_prox -= 1
                elif direcao == "b": pos_i_prox += 1

                # Verifica se está nos limites do tabuleiro
                if not (0 <= pos_i_prox < tam_i and 0 <= pos_j_prox < tam_j):
                    # Colisão com a parede
                    recompensa_teste -= 10
                    pos_i_prox, pos_j_prox = pos_i, pos_j
                else:
                    pos_i, pos_j = pos_i_prox, pos_j_prox

            elif acao.startswith("atirar") and flecha:
                flecha = False
                recompensa_teste -= 10 # Custo de atirar

                target_i, target_j = pos_i, pos_j
                direcao = acao.split(" ")[1]
                if direcao == "d": target_j += 1
                elif direcao == "e": target_j -= 1
                elif direcao == "c": target_i -= 1
                elif direcao == "b": target_i += 1

                if 0 <= target_i < tam_i and 0 <= target_j < tam_j:
                    if tabuleiro[target_i][target_j][0] == 2: # Acertou
                        recompensa_teste += 500
                        wumpus_morto = True
                        # Retira o Wumpus do tabuleiro
                        edita_wumpus(tabuleiro, tam_i, tam_j, tam_k, target_i, target_j, 0, 0, 0)

            elif acao == "pegar ouro":
                if tabuleiro[pos_i][pos_j][2] == 6 and not pegou_ouro:
                    pegou_ouro = True
                    recompensa_teste += 1000
                    tabuleiro[pos_i][pos_j][2] = 0
                elif pegou_ouro:
                    recompensa_teste -= 5
                else:
                    recompensa_teste -= 5

            elif acao == "sair caverna":
                if pos_i == tam_i - 1 and pos_j == 0 and pegou_ouro:
                    recompensa_teste += 1000
                    done = True
                    num_vitorias +=1
                else:
                    recompensa_teste -= 10

            # Verifica mortes
            if not done:
                if tabuleiro[pos_i][pos_j][0] == 2 and not wumpus_morto:
                    recompensa_teste -= 1000
                    done = True
                elif tabuleiro[pos_i][pos_j][1] == 4:
                    recompensa_teste -= 1000
                    done = True

            recompensa_atual += recompensa_teste

            # Observa o próximo estado
            cheiro_ruim_prox = (tabuleiro[pos_i][pos_j][0] == 3)
            brisa_prox = (tabuleiro[pos_i][pos_j][1] == 5)
            brilho_prox = (tabuleiro[pos_i][pos_j][2] == 6)
            percepcoes_proximo = (cheiro_ruim_prox, brisa_prox, brilho_prox)

            prox_estado = informacao_estado(pos_i, pos_j, percepcoes_proximo, flecha, pegou_ouro)

            # Atualiza a tabela
            estado_para_atualizar = informacao_estado(pos_i_velha, pos_j_velha, percepcoes_atual, flecha_velha, pegou_ouro_velha)
            atualizar_tabela(estado_para_atualizar, acao, recompensa_teste, prox_estado, done, flecha, pegou_ouro, pos_i, pos_j)

            iteracao += 1

        recompensa_episodio.append(recompensa_atual)
        epsilon_atual = epsilon_atual * epsilon_decay

        if (episodio + 1) % 100 == 0:
            recompensa_media = sum(recompensa_episodio[-100:]) / 100
            print(f"Episodio {episodio + 1}/{numero_episodios} | Recompensa med. (ultimos 100): {recompensa_media:.2f} | epsilon: {epsilon_atual:.3f} | Vitórias (ultimos 100): {num_vitorias}")
            num_vitorias = 0

    print("Fim do treinamento")

    # Resultado
    print("\n--- Executando o agente treinado ---")
    # Prepara o ambiente novamente
    tabuleiro = [[[0 for _ in range(tam_k)] for _ in range(tam_j)] for _ in range(tam_i)]
    wumpus_i, wumpus_j = preenche_wumpus(tabuleiro, tam_i, tam_j, tam_k, 0, 2, 3)
    pit_coords_list = preenche_abismo(tabuleiro, tam_i, tam_j, tam_k, 1, 4, 5, (wumpus_i, wumpus_j))
    ouro_i, ouro_j = preenche_ouro(tabuleiro, tam_i, tam_j, tam_k, 2, 6, (wumpus_i, wumpus_j), pit_coords_list)

    pos_i = tam_i - 1
    pos_j = 0
    flecha = True
    pegou_ouro = False
    wumpus_morto_teste = False
    done_teste = False
    passos = 0
    maximo_passos = 50
    recompensa_teste = 0

    print(f"Começando na posição [{tam_i-pos_i}, {pos_j+1}]")
    print("Tabuleiro inicial com o agente, abismos, Wumpus e ouro")
    for r_idx, row_val in enumerate(tabuleiro):
        display_row = []
        for c_idx, cell_val in enumerate(row_val):
            s = ""
            if cell_val[0] == 2: s += "W"
            if cell_val[1] == 4: s += "A"
            if cell_val[2] == 6: s += "O"
            if not s: s = "."
            if r_idx == pos_i and c_idx == pos_j: s = "A"
            display_row.append(s.ljust(3))
        print(" ".join(display_row))
    print("-" * 20)


    while not done_teste and passos < maximo_passos:
        cheiro_ruim = (tabuleiro[pos_i][pos_j][0] == 3)
        brisa = (tabuleiro[pos_i][pos_j][1] == 5)
        brilho = (tabuleiro[pos_i][pos_j][2] == 6)
        percepcoes_atual = (cheiro_ruim, brisa, brilho)
        state = informacao_estado(pos_i, pos_j, percepcoes_atual, flecha, pegou_ouro)

        acao = define_acao(state, flecha, pegou_ouro, pos_i, pos_j, 0.0)

        print(f"Ação {passos+1}: {acao} | Estado=([{pos_i},{pos_j}],CR:{cheiro_ruim},BS:{brisa},BO:{brilho},F:{flecha},PO:{pegou_ouro})")

        recompensa_acao = -1
        if acao.startswith("mover"):
            direcao = acao.split(" ")[1]
            pos_i_prox_teste, pos_j_prox_teste = pos_i, pos_j
            if direcao == "d": pos_j_prox_teste += 1
            elif direcao == "e": pos_j_prox_teste -= 1
            elif direcao == "c": pos_i_prox_teste -= 1
            elif direcao == "b": pos_i_prox_teste += 1

            if not (0 <= pos_i_prox_teste < tam_i and 0 <= pos_j_prox_teste < tam_j):
                print("Simulador: Bateu na parede!")
                recompensa_acao -=10
            else:
                pos_i, pos_j = pos_i_prox_teste, pos_j_prox_teste

        elif acao.startswith("atirar") and flecha:
            flecha = False
            recompensa_acao -=10
            target_i, target_j = pos_i, pos_j
            direcao = acao.split(" ")[1]
            if direcao == "d": target_j += 1
            elif direcao == "e": target_j -= 1
            elif direcao == "c": target_i -= 1
            elif direcao == "b": target_j += 1

            if 0 <= target_i < tam_i and 0 <= target_j < tam_j:
                if tabuleiro[target_i][target_j][0] == 2 and not wumpus_morto_teste:
                    print("Simulador: O Wumpus é morto e solta um grito!")
                    wumpus_morto_teste = True
                    recompensa_acao += 500
                    edita_wumpus(tabuleiro, tam_i, tam_j, tam_k, target_i, target_j, 0, 0, 0)
                else:
                    print("Simulador: Você ouve o som da flecha caindo no chão...")
            else:
                 print("Simulador: Você ouve o som da flecha caindo no chão...")


        elif acao == "pegar ouro":
            if tabuleiro[pos_i][pos_j][2] == 6 and not pegou_ouro:
                pegou_ouro = True
                tabuleiro[pos_i][pos_j][2] = 0
                print("Simulador: Você pegou o ouro! Agora pode sair da caverna!")
                recompensa_acao +=1000
            elif pegou_ouro: print("Simulador: Você já está com o ouro.")
            else: print("Simulador: O ouro não está aqui... Procure pelo seu brilho.")

        elif acao == "sair caverna":
            if pos_i == tam_i - 1 and pos_j == 0 and pegou_ouro:
                print("Simulador: Saiu da caverna com ouro!")
                done_teste = True
                recompensa_acao +=1000
            else:
                print("Simulador: Para sair é preciso ter pego o ouro e estar na posição [1, 1].")
                recompensa_acao -=10

        # Verifica morte
        if not done_teste:
            if tabuleiro[pos_i][pos_j][0] == 2 and not wumpus_morto_teste:
                print(f"Simulador: MORTO PELO WUMPUS em [{tam_i-pos_i}, {pos_j+1}]")
                done_teste = True
                recompensa_acao -=1000
            elif tabuleiro[pos_i][pos_j][1] == 4:
                print(f"Simulador: CAIU NO ABISMO em [{tam_i-pos_i}, {pos_j+1}]")
                done_teste = True
                recompensa_acao -=1000

        recompensa_teste += recompensa_acao
        passos += 1
        print(f"Nova Posição: [{tam_i-pos_i}, {pos_j+1}], Recompensa da ação: {recompensa_acao}, Total: {recompensa_teste}\n")
        if done_teste:
            print("Fim do teste")

    if not done_teste:
        print("Maximo de ações alcançado")

Episodio 100/5000 | Recompensa med. (ultimos 100): -1015.22 | epsilon: 0.905 | Vitórias (ultimos 100): 0
Episodio 200/5000 | Recompensa med. (ultimos 100): -971.94 | epsilon: 0.819 | Vitórias (ultimos 100): 0
Episodio 300/5000 | Recompensa med. (ultimos 100): -865.03 | epsilon: 0.741 | Vitórias (ultimos 100): 0
Episodio 400/5000 | Recompensa med. (ultimos 100): -785.53 | epsilon: 0.670 | Vitórias (ultimos 100): 0
Episodio 500/5000 | Recompensa med. (ultimos 100): -716.51 | epsilon: 0.606 | Vitórias (ultimos 100): 0
Episodio 600/5000 | Recompensa med. (ultimos 100): -696.18 | epsilon: 0.549 | Vitórias (ultimos 100): 2
Episodio 700/5000 | Recompensa med. (ultimos 100): -475.59 | epsilon: 0.496 | Vitórias (ultimos 100): 2
Episodio 800/5000 | Recompensa med. (ultimos 100): -108.18 | epsilon: 0.449 | Vitórias (ultimos 100): 16
Episodio 900/5000 | Recompensa med. (ultimos 100): 24.85 | epsilon: 0.406 | Vitórias (ultimos 100): 16
Episodio 1000/5000 | Recompensa med. (ultimos 100): 288.72 | ep